### Health Check

In [1]:
[ordered]@{"$((Get-TimeZone).Id)" = Get-Date
    'Boot'      = (Get-CimInstance win32_operatingsystem).lastbootuptime.ToString('ddd M.d.yy H:m')
    'CpuUsage%' = (Get-CimInstance win32_processor | Measure-Object LoadPercentage -Average).Average
    'RamUsage%' = {$os=Get-CimInstance win32_operatingsystem; [Math]::Ceiling(($os.TotalVisibleMemorySize-$os.FreePhysicalMemory)*100/$os.TotalVisibleMemorySize)}.Invoke()
    'C:%Free'   = { $disk = Get-CimInstance Win32_LogicalDisk -Filter "DeviceID='C:'"; [Math]::Ceiling(($disk.FreeSpace / $disk.Size) * 100) }.Invoke()
            } | ft -HideTableHeaders -AutoSize


Central Standard Time 12/3/2023 1:05:17 PM
Boot                  Sun 12.3.23 11:9
CpuUsage%             88
RamUsage%             {55}
C:%Free               {55}



This shows which processes are consuming the most cpu, memory and network and have the most number of handles and threads currently on the system.

In [4]:
$objArr = Get-Process | Select-Object @{Name='Process'; Expression={$_.ProcessName+'►'+$_.Id}}, CPU, ID, WS, HandleCount, Threads
# Sort for each property separately
$sortedByCPU = $objArr | Sort-Object CPU -Descending | Select-Object -First 10
$sortedByWS = $objArr | Sort-Object WS -Descending | Select-Object -First 10
$sortedByHandles = $objArr | Sort-Object HandleCount -Descending | Select-Object -First 10
$sortedByThreads = $objArr | Sort-Object Threads -Descending | Select-Object -First 10

$Network = (Get-Counter '\Process(*)\IO Read Operations/sec').CounterSamples.Where({$_.InstanceName -ne "_Total" -and $_.InstanceName -ne "Idle"})|
                sort cookedvalue -Descending|Select -Property instancename -First 10

$harray = for ($j = 0; $j -lt 10; $j++) {
    [PSCustomObject][ordered]@{
        'CPU'     = $sortedByCPU[$j].Process
        'WS'      = $sortedByWS[$j].Process
        'Handles' = $sortedByHandles[$j].Process
        'Threads' = $sortedByThreads[$j].Process
        'Network' = $Network[$j].InstanceName
    }
}
$harray | Format-Table -AutoSize


CPU                     WS                      Handles                 Threads                 Network
---                     --                      -------                 -------                 -------
msedge►13512            dotnet►8888             System►4                System►4                code - insiders
msedge►7464             msedge►7464             explorer►6532           Memory Compression►2632 code - insiders
Grammarly.Desktop►12308 dotnet►9916             msedge►7464             explorer►6532           code - insiders
OneDrive►11916          dwm►772                 lsass►904               SearchHost►6964         code - insiders
msedge►6124             Grammarly.Desktop►12308 OneDrive►11916          msedge►7464             code - insiders
Code - Insiders►1996    Code - Insiders►6892    Grammarly.Desktop►12308 Code - Insiders►12716   svchost
explorer►6532           Code - Insiders►11536   SearchHost►6964         msedgewebview2►14164    code - insiders
msedge►9736    